# Bitcoin Reaction to Macroeconomic Events with SP500, VIX, and Treasury Controls

This notebook integrates Bitcoin price data, SP500 returns, VIX changes, and 10-year Treasury yield changes to conduct a Difference-in-Differences regression analysis.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from arch import arch_model

## Load BTC Data and Calculate Log Returns

In [2]:
# Load BTC data (control + event periods)
btc_control = pd.read_csv("../data/btc_control_period_clean.csv", parse_dates=['date'])
btc_event = pd.read_csv("../data/btc_event_period_clean.csv", parse_dates=['date'])

# Calculate log returns
btc_control['log_return'] = np.log(btc_control['close'] / btc_control['close'].shift(1))
btc_event['log_return'] = np.log(btc_event['close'] / btc_event['close'].shift(1))

# Combine event and control periods
btc_all = pd.concat([btc_control, btc_event])
btc_all['event_period'] = btc_all['date'].apply(lambda d: 1 if d in btc_event['date'].values else 0)

## Load SP500 and VIX Data

In [11]:
# Load SP500 and VIX data
sp500_control = pd.read_csv("../data/sp500_control_period_clean.csv", parse_dates=["date"])
sp500_event = pd.read_csv("../data/sp500_event_period_clean.csv", parse_dates=["date"])
vix_control = pd.read_csv("../data/vix_control_period_clean.csv", parse_dates=["date"])
vix_event = pd.read_csv("../data/vix_event_period_clean.csv", parse_dates=["date"])

# Combine control + event
sp500_all = pd.concat([sp500_control, sp500_event])
vix_all = pd.concat([vix_control, vix_event])

sp500_all.rename(columns={"close": "sp500_close"}, inplace=True)
vix_all.rename(columns={"close": "vix_close"}, inplace=True)

# Compute daily percentage changes
sp500_all['sp500_return'] = sp500_all['sp500_close'].pct_change()
vix_all['vix_change'] = vix_all['vix_close'].pct_change()

## Load Treasury Yield Data

In [17]:
# Load 10-year Treasury yield data
treasury_control = pd.read_csv("../data/treasury_yield_control_period_clean"
".csv", parse_dates=["date"])
treasury_event = pd.read_csv("../data/treasury_yield_event_period_clean.csv", parse_dates=["date"])

# Combine control + event
treasury_all = pd.concat([treasury_control, treasury_event])

# Calculate daily yield changes
treasury_all['yield_change'] = treasury_all['treasury_yield'].diff()

## Merge Control Variables with BTC Data

In [18]:
# Merge SP500, VIX, and Treasury yields into BTC dataset
btc_all = btc_all.merge(sp500_all[['date', 'sp500_return']], on='date', how='left')
btc_all = btc_all.merge(vix_all[['date', 'vix_change']], on='date', how='left')
btc_all = btc_all.merge(treasury_all[['date', 'yield_change']], on='date', how='left')

# Drop rows with missing values
btc_all.dropna(subset=['log_return', 'sp500_return', 'vix_change', 'yield_change'], inplace=True)

btc_all.head()

,date,open,high,low,close,volume,log_return,event_period,sp500_return,vix_change,yield_change
1,2024-08-21,59027.55,61851.26,58806.66,61170.80,2.311932e+09,0.035666,0,0.003437,0.015595,-0.03
2,2024-08-22,61170.80,61426.96,59756.43,60387.29,1.754743e+09,-0.012891,0,-0.007848,0.042120,0.07
3,2024-08-23,60387.29,64982.66,60351.28,64087.80,2.623908e+09,0.059475,0,0.010625,-0.059859,-0.05
6,2024-08-26,64264.62,64504.12,62816.08,62850.27,1.719932e+09,-0.022254,0,-0.002384,-0.000871,0.01
7,2024-08-27,62850.27,63220.85,58045.32,59441.73,2.403744e+09,-0.055759,0,0.001373,-0.019826,0.01


## Difference-in-Differences Regression with SP500, VIX, and Treasury Controls

In [19]:
# Define regression model with event_period, SP500, VIX, and Treasury yield changes as controls
X = sm.add_constant(btc_all[['event_period', 'sp500_return', 'vix_change', 'yield_change']])
y = btc_all['log_return']

model = sm.OLS(y, X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             log_return   R-squared:                       0.299
Model:                            OLS   Adj. R-squared:                  0.227
Method:                 Least Squares   F-statistic:                     4.165
Date:                Fri, 21 Mar 2025   Prob (F-statistic):            0.00664
Time:                        00:13:08   Log-Likelihood:                 102.37
No. Observations:                  44   AIC:                            -194.7
Df Residuals:                      39   BIC:                            -185.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.0015      0.005      0.283      0.779      -0.009       0.013
event_period     0.0065      0.008      0.831      0.411      -0.009       0.022
sp500_return     1.6765      0.658      2.549      0.015       0.346       3.007
vix_change      -0.0199      0.111     -0.179      0.859      -0.245       0.205
yield_change    -0.1054      0.060     -1.757      0.087      -0.227       0.016
==============================================================================
Omnibus:                        0.572   Durbin-Watson:                   2.336
Prob(Omnibus):                  0.751   Jarque-Bera (JB):                0.259
Skew:                           0.187   Prob(JB):                        0.878
Kurtosis:                       3.036   Cond. No.                         201.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## GARCH(1,1) Volatility Modeling

In [20]:
# Fit a GARCH(1,1) model to examine volatility dynamics
returns = btc_all['log_return'] * 100  # scale to percentage
garch_model = arch_model(returns, vol='Garch', p=1, q=1)
garch_results = garch_model.fit(disp='off')
print(garch_results.summary())

                     Constant Mean - GARCH Model Results                      
Dep. Variable:             log_return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -107.888
Distribution:                  Normal   AIC:                           223.776
Method:            Maximum Likelihood   BIC:                           230.913
                                        No. Observations:                   44
Date:                Fri, Mar 21 2025   Df Residuals:                       43
Time:                        00:13:16   Df Model:                            1
                               Mean Model                               
                 coef    std err          t      P>|t|  95.0% Conf. Int.
------------------------------------------------------------------------
mu             0.5668      0.418      1.355      0.175 [ -0.253,  1.38